In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trying2").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 14:47:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [4]:
import pp_events as pp

In [5]:
events_shot = pp.preprocessing(events)

In [6]:
events_shot.show()

+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+
|                  id|player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|goal|
+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+
|c7168289-8ba6-427...|  30111.0|          111.6|           34.7|          9.9323| 39.113426|                  0|             1|              1|              0|              0|             0|                   0|       0.05332383|       Off T|   0|
|aa63a13

In [7]:
#features = ['distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure']
features = [#'shot_location_x', 'shot_location_y',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble']
target = ['goal']

In [8]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)

In [9]:
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [10]:
from pyspark.ml.classification import LogisticRegression

# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)

In [11]:
from pyspark.ml import Pipeline

# Create the pipeline
pipeline = Pipeline(stages=[log_reg])

In [12]:
xg_model = pipeline.fit(train_data)

24/12/23 14:49:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [13]:
# Make predictions
predictions = xg_model.transform(test_data)

# Evaluate using BinaryClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.7820792751956707


In [14]:
predictions.show(truncate=False)

+------------------------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+--------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|id                                  |player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|goal|features_vector                                   |rawPrediction                           |probability                              |prediction|
+------------------------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+-------

In [15]:
from pyspark.sql.functions import col, udf, format_number, abs
from pyspark.sql.types import DoubleType

# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference_abs", abs(col('difference')))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference",'difference_abs').show(50,False)

+-----------------+----------------+-------------+--------------+
|shot_statsbomb_xg|goal_probability|difference   |difference_abs|
+-----------------+----------------+-------------+--------------+
|0.10166888       |0.2016554697    |-0.0999865897|0.0999865897  |
|0.064399794      |0.1741354923    |-0.1097356983|0.1097356983  |
|0.033228997      |0.0884812211    |-0.0552522241|0.0552522241  |
|0.0447311        |0.1285305264    |-0.0837994264|0.0837994264  |
|0.10890295       |0.1488687279    |-0.0399657779|0.0399657779  |
|0.03492738       |0.1081572712    |-0.0732298912|0.0732298912  |
|0.0638635        |0.0830193094    |-0.0191558094|0.0191558094  |
|0.24384445       |0.0866435851    |0.1572008649 |0.1572008649  |
|0.033889998      |0.0781243248    |-0.0442343268|0.0442343268  |
|0.30627033       |0.1435808248    |0.1626895052 |0.1626895052  |
|0.038757622      |0.0740279974    |-0.0352703754|0.0352703754  |
|0.030633146      |0.0219802906    |0.0086528554 |0.0086528554  |
|0.1491743

In [16]:
# average of the difference
from pyspark.sql.functions import avg

avg_diff = predictions_with_goal_prob.select(avg(col("difference"))).collect()[0][0]
print(f"Average difference: {avg_diff}")

abs_avg_diff = predictions_with_goal_prob.select(avg(col("difference_abs"))).collect()[0][0]
print(f"Abs average difference: {abs_avg_diff}")

Average difference: -0.0035195385080439936


Abs average difference: 0.052823549481646526


In [22]:
# Average difference    : -0.005542601315472378
# Abs average difference:  0.05229424852629077

# Without x y location 
# Average difference    : -0.0035195385080439936
# Abs average difference:  0.052823549481646526

In [17]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model

from pyspark.sql.functions import round

predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

predictions_with_goal_prob.select("goal", "goal_probability_rounded", "shot_statsbomb_xg_rounded").show()

+----+------------------------+-------------------------+
|goal|goal_probability_rounded|shot_statsbomb_xg_rounded|
+----+------------------------+-------------------------+
|   0|                     0.0|                      0.0|
|   1|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   1|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   1|        

In [18]:
from pyspark.sql.functions import col, when

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0)
).withColumn(
    "statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)

In [19]:
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")

Model Accuracy: 89.41%
StatsBomb xG Accuracy: 90.42%


In [ ]:
# Model Accuracy: 89.40%
# StatsBomb xG Accuracy: 90.42%

# Without x and y
# Model Accuracy: 89.41%
# StatsBomb xG Accuracy: 90.42%

In [23]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

shot_location_x: -0.07089985223630016
shot_location_y: -0.004657175090242295
distance_to_goal: -0.2131395419906175
shot_angle: 0.0119234592952614
preferred_foot_shot: 0.9855072780022439
under_pressure: -0.08668524824306786
shot_aerial_won: -1.1647390169920981
shot_first_time: -0.3580628236898425
shot_one_on_one: 0.41576264305295624
shot_open_goal: 1.4443852791674474
shot_follows_dribble: 1.5189010520536304


In [20]:
# Without X Y
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

distance_to_goal: -0.11629356522121186
shot_angle: 0.02124864814005136
preferred_foot_shot: 0.9045120912520695
under_pressure: -0.1658760201803778
shot_aerial_won: -1.1013926945551096
shot_first_time: -0.2703307606218668
shot_one_on_one: 0.37407505616181824
shot_open_goal: 1.2061576442392097
shot_follows_dribble: 1.1792647451380696


In [21]:
# Testing matching of models, is out model as good as SB_xg?
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_to_sbxg", when(col("goal_probability_rounded") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)

In [22]:
xg_accuracy_model = predictions_with_accuracy.selectExpr("avg(model_to_sbxg) as model_accuracy").first()["model_accuracy"]

print(f"Models matching percentage: {xg_accuracy_model * 100:.2f}%")

Models matching percentage: 97.17%
